In [19]:
import os
from google import genai
from dotenv import load_dotenv
from pypdf import PdfReader
import markdown
from unstructured.partition.text import partition_text
import chromadb
from chromadb.utils import embedding_functions

## cfg

In [20]:

currentdir = os.getcwd()
input_dir = "../data/1020test/raw" # 入力ディレクトリ
persist_db_dir = "../data/1020test/vector_store" # データベースの保存ディレクトリ
collection_name = "1020_test" # コレクション名


In [ ]:
# .envファイルからAPIキーを読み込む
env_path = os.path.join(currentdir, '..', 'configs', '.env')
load_dotenv(dotenv_path=env_path)

GEMINI_API_KEY = os.getenv("geminiapikey")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEYが見つかりません。../configs/.envファイルを確認してください。")



In [23]:
chat_model = "gemini-2.5-flash"
embed_model = "gemini-embedding-001"
client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
### api access check
response = client.models.generate_content(
    model=chat_model, contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make informed predictions or decisions.


---

## test用コード  
こちらは検証用にデータベース蓄積用の文書ファイルを仮で作成するためのコード

In [ ]:

# input_dirが存在しない場合は作成し、ダミーファイルを生成
if not os.path.exists(input_dir):
    os.makedirs(input_dir)
    print(f"'{input_dir}' ディレクトリを作成しました。この中にPDFやMarkdownファイルを入れてください。")
    print("ダミーファイルを生成します...")

    md_example_path = os.path.join(input_dir, "example_doc1.md")
    md_example_path_2 = os.path.join(input_dir, "example_doc2.md")


    with open(md_example_path, "w", encoding="utf-8") as f:
        f.write("# Gemini APIの概要\n\n")
        f.write("Gemini APIは、Googleの最新のマルチモーダルAIモデルであるGeminiモデルへのアクセスを提供します。\n")
        f.write("これは、テキスト、画像、音声、動画などの異なるタイプの情報を理解し、処理することができます。\n\n")
        f.write("## 主な機能\n\n")
        f.write("*   **柔軟なモデル:** さまざまな規模のモデル（Ultra, Pro, Nano）があります。\n")
        f.write("*   **マルチモーダル:** テキストだけでなく、画像などの入力も扱えます。\n")
        f.write("*   **高度な推論:** 複雑な問題解決能力を持っています。\n\n")
        f.write("RAG (Retrieval Augmented Generation) は、外部知識ベースから情報を取得し、その情報に基づいて応答を生成するAIシステムです。\n")
        f.write("これにより、モデルはより正確で最新の情報を利用できます。質問応答システムによく用いられます。\n")

    with open(md_example_path_2, "w", encoding="utf-8") as f:
        f.write("# ベクトルデータベースについて\n\n")
        f.write("ベクトルデータベースは、データを高次元のベクトル表現として保存し、類似性検索を効率的に行うためのデータベースです。\n")
        f.write("特にAIや機械学習の分野で、意味的に関連する情報を素早く見つけ出すのに役立ちます。\n\n")
        f.write("## 利点\n\n")
        f.write("*   **高速な類似性検索:** ユークリッド距離やコサイン類似度などの指標で類似度を計算します。\n")
        f.write("*   **スケーラビリティ:** 大規模なデータセットにも対応できます。\n")
        f.write("*   **セマンティック検索:** キーワードだけでなく、意味に基づいた検索が可能です。\n")
        f.write("ChromaDBは、Pythonで簡単に利用できる軽量なベクトルデータベースであり、RAGシステムの実装に適しています。\n")
    print("ダミーファイルの生成が完了しました。")
else:
    print(f"'{input_dir}' ディレクトリは既に存在します。既存のファイルを使用します。")

'../data/1020test/raw' ディレクトリを作成しました。この中にPDFやMarkdownファイルを入れてください。
ダミーファイルを生成します...
ダミーファイルの生成が完了しました。


---

## 関数

In [26]:

# --- ドキュメントの読み込みとチャンク化 ---

def load_document(file_path):
    """ファイルパスに基づいてPDFまたはMarkdownファイルを読み込む"""
    _, ext = os.path.splitext(file_path)
    text_content = ""

    if ext.lower() == ".pdf":
        try:
            reader = PdfReader(file_path)
            for page in reader.pages:
                text_content += page.extract_text() if page.extract_text() else ""
        except Exception as e:
            print(f"PDFファイル '{file_path}' の読み込み中にエラーが発生しました: {e}")
            return None
    elif ext.lower() == ".md":
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                md_content = f.read()
                text_content = markdown.markdown(md_content) 
                elements = partition_text(text=text_content, chunk_elements_by_title=False)
                text_content = "\n".join([str(el) for el in elements])
        except Exception as e:
            print(f"Markdownファイル '{file_path}' の読み込み中にエラーが発生しました: {e}")
            return None
    else:
        print(f"サポートされていないファイル形式です: {file_path}")
        return None
    return text_content

def chunk_text(text, chunk_size=1000, overlap_size=100):
    """テキストをチャンクに分割する"""
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        if end >= len(text):
            break
        start += (chunk_size - overlap_size)
    return chunks


In [27]:

# --- ベクトルデータベースの作成と保存 (ChromaDB) ---

def create_vector_database(documents_data, collection_name="my_rag_collection", persist_directory="./chroma_db"):
    """
    複数のドキュメントからベクトルデータベースを作成し、永続化する
    documents_data は [(ファイルパス, テキストコンテンツ), ...] のリスト
    """
    all_chunks = []
    all_ids = []
    all_metadatas = []
    global_chunk_id = 0

    print(f"ベクトルデータベース '{collection_name}' を作成中...")

    client = chromadb.PersistentClient(path=persist_directory)
    gemini_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
        api_key=GEMINI_API_KEY, model_name=embed_model
    )
    collection = client.get_or_create_collection(name=collection_name, embedding_function=gemini_ef)

    current_doc_count = collection.count()
    print(f"既存のコレクション '{collection_name}' には {current_doc_count} 個のドキュメントがあります。")
    
    for file_path, full_text in documents_data:
        if full_text:
            chunks = chunk_text(full_text)
            print(f"ファイル '{file_path}' を {len(chunks)} 個のチャンクに分割しました。")
            for i, chunk in enumerate(chunks):
                all_chunks.append(chunk)
                all_ids.append(f"doc_chunk_{global_chunk_id}")
                all_metadatas.append({"source": os.path.basename(file_path), "chunk_id": i, "file_path": file_path})
                global_chunk_id += 1
        else:
            print(f"ファイル '{file_path}' のテキストコンテンツが空のため、スキップします。")

    if not all_chunks:
        print("処理する新しいチャンクが見つかりませんでした。既存のデータベースを使用します。")
        return collection

    print(f"{len(all_chunks)} 個のチャンクを埋め込み、データベースに追加中...")
    collection.add(
        documents=all_chunks,
        metadatas=all_metadatas,
        ids=all_ids
    )
    print(f"ベクトルデータベース '{collection_name}' が '{persist_directory}' に正常に更新されました。現在のドキュメント数: {collection.count()}")
    return collection


In [30]:

# --- RAGチャットの実装 ---

def rag_chat(collection, query, top_k=3):
    """
    RAG (Retrieval Augmented Generation) を用いた単一応答機能
    """
    if collection.count() == 0:
        return "データベースが空のため、質問に答えることができません。"

    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )

    retrieved_documents = results['documents'][0]
    if not retrieved_documents:
        return "関連するドキュメントが見つかりませんでした。質問の仕方を変えてみてください。"

    context = "\n".join(retrieved_documents)
    
    print("\n--- 検索されたドキュメント ---")
    for i, doc in enumerate(retrieved_documents):
        print(f"ドキュメント {i+1} (ソース: {results['metadatas'][0][i]['source']}):")
        print(doc[:200].replace('\n', ' ') + "...") 
        print("-" * 20)

    prompt = f"""以下の情報に基づいて質問に答えてください。質問に関連しない情報は無視してください。
    情報:
    {context}

    質問: {query}
    """


    print("\n--- Geminiモデルが応答を生成中 ---")
    try:
        response = client.models.generate_content(
            model=chat_model, 
            contents=prompt
            )

        return response.text
    except Exception as e:
        return f"Geminiモデルからの応答生成中にエラーが発生しました: {e}"


## run

### データベースの作成

In [28]:

documents_to_process = []
print(f"ディレクトリ '{input_dir}' 内のファイルを検索中...")
for filename in os.listdir(input_dir):
    file_path = os.path.join(input_dir, filename)
    if os.path.isfile(file_path):
        if filename.lower().endswith(('.pdf', '.md')):
            print(f"ファイル '{filename}' を読み込み中...")
            content = load_document(file_path)
            if content:
                documents_to_process.append((file_path, content))
            else:
                print(f"警告: ファイル '{file_path}' の読み込みに失敗したか、内容が空です。")
        else:
            print(f"スキップ: サポートされていないファイル形式 '{filename}'")

if not documents_to_process:
    print(f"'{input_dir}' ディレクトリに処理可能なPDFまたはMarkdownファイルが見つかりませんでした。")
    unified_collection = None # データベースが作成できなかったことを示す
else:
    print("\n--- 複数のドキュメントからベクトルデータベースを作成または更新 ---")
    unified_collection = create_vector_database(documents_to_process, collection_name=collection_name, persist_directory=persist_db_dir)
    print("ベクトルデータベースの準備が完了しました。")


ディレクトリ '../data/1020test/raw' 内のファイルを検索中...
ファイル 'example_doc2.md' を読み込み中...


Exception ignored in: <function Client.__del__ at 0x000001E1AC6D0180>
Traceback (most recent call last):
  File "c:\Users\ryoic\repository\251012_dev.git\.venv\Lib\site-packages\google\genai\client.py", line 400, in __del__
    self.close()
  File "c:\Users\ryoic\repository\251012_dev.git\.venv\Lib\site-packages\google\genai\client.py", line 386, in close
    self._api_client.close()
    ^^^^^^^^^^^^^^^^
AttributeError: 'Client' object has no attribute '_api_client'


ファイル 'example_doc3.md' を読み込み中...

--- 複数のドキュメントからベクトルデータベースを作成または更新 ---
ベクトルデータベース '1020_test' を作成中...


c:\Users\ryoic\repository\251012_dev.git\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


既存のコレクション '1020_test' には 0 個のドキュメントがあります。
ファイル '../data/1020test/raw\example_doc2.md' を 1 個のチャンクに分割しました。
ファイル '../data/1020test/raw\example_doc3.md' を 1 個のチャンクに分割しました。
2 個のチャンクを埋め込み、データベースに追加中...
ベクトルデータベース '1020_test' が '../data/1020test/vector_store' に正常に更新されました。現在のドキュメント数: 2
ベクトルデータベースの準備が完了しました。


In [ ]:

if unified_collection and unified_collection.count() > 0:
    print("\n--- 質問と応答 ---")
    # ここに質問文を入力してください
    user_query = "Gemini APIの主な機能は何ですか？RAGについても説明してください。" 
    
    response = rag_chat(unified_collection, user_query)
    print(f"\n質問: {user_query}")
    print(f"応答: {response}")
else:
    print("ベクトルデータベースが利用できません。データベースを作成または更新してください。")


--- 質問と応答 ---

--- 検索されたドキュメント ---
ドキュメント 1 (ソース: example_doc2.md):
<h1>Gemini APIの概要</h1> <p>Gemini APIは、Googleの最新のマルチモーダルAIモデルであるGeminiモデルへのアクセスを提供します。 これは、テキスト、画像、音声、動画などの異なるタイプの情報を理解し、処理することができます。</p> <h2>主な機能</h2> <ul> <li><strong>柔軟なモデル:</strong> さまざまな規模のモデル（Ult...
--------------------
ドキュメント 2 (ソース: example_doc3.md):
<h1>ベクトルデータベースについて</h1> <p>ベクトルデータベースは、データを高次元のベクトル表現として保存し、類似性検索を効率的に行うためのデータベースです。 特にAIや機械学習の分野で、意味的に関連する情報を素早く見つけ出すのに役立ちます。</p> <h2>利点</h2> <ul> <li><strong>高速な類似性検索:</strong> ユークリッド距離やコサイン類似度などの指標...
--------------------

--- Geminiモデルが応答を生成中 ---

質問: Gemini APIの主な機能は何ですか？RAGについても説明してください。
応答: Gemini APIの主な機能は以下の通りです。

*   **柔軟なモデル:** さまざまな規模のモデル（Ultra, Pro, Nano）があります。
*   **マルチモーダル:** テキストだけでなく、画像などの入力も扱えます。
*   **高度な推論:** 複雑な問題解決能力を持っています。

RAG (Retrieval Augmented Generation) は、外部知識ベースから情報を取得し、その情報に基づいて応答を生成するAIシステムです。これにより、モデルはより正確で最新の情報を利用できます。質問応答システムによく用いられます。
